# 03-stack_rasters

From Sentinel (Copernicus) 10 dates from Oct2024 to Oct2025 were downloaded.

This notebook calculates the mean value from all downloaded rasters by band and saves them localy.

## __Import libraries__

In [1]:
from pathlib import Path
current_path = Path().resolve()
for parent in current_path.parents:
    if parent.name == "accesibilidad-urbana":
        project_root = parent
        break
print(project_root)

/home/jovyan/accesibilidad-urbana


In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

import rasterio as ro

import matplotlib.pyplot as plt
import seaborn as sns

## __Config notebook__

In [3]:
dir_grl = str(project_root)+'/data/processed/2025_caracterizacion_forestal/downloaded_sentinel/'
print(dir_grl)

bands_to_join = ['B02','B03','B04','B05','B06','B07','B8A','B11','B12']

raster_dates = ['20241126T170641', #Nov 2024
                '20241226T170731', #Dec 2024
                '20250115T170641', #Jan 2025
                '20250209T170349', #Feb 2025
                '20250318T170711', #Mar 2025
                '20250410T165849', #Apr 2025
                '20250515T165901', #May 2025
                #'20250609T165849', #Jun 2025 (Removed because has clouds on the west aoi)
                '20250803T165911', #Ago 2025
                #'20251012T170251' #Oct 2025 (Removed because has clouds on the west aoi)
               ]

/home/jovyan/accesibilidad-urbana/data/processed/2025_caracterizacion_forestal/downloaded_sentinel/


## __Function definition__

In [5]:
def stacked_arrays(raster_band):
    # This function stacks all current raster_band available raster dates.

    print(f'{raster_band} - Stacking arrays.')
    
    arrays = []
    stacked_arrays = []
    
    for date in raster_dates:
        with ro.open(dir_grl+f"{date}/T14QMJ_{date}_{raster_band}_20m.jp2", "r") as ds:
            img_tmp = ds.read()
            img_tmp = img_tmp.reshape((img_tmp.shape[1],img_tmp.shape[2]))
        
        arrays.append(img_tmp)
    
    for array in arrays:
        stacked_array = np.stack((array,) + tuple(stacked_arrays))
        stacked_arrays.append(array)
    
    img_stacked = np.array(stacked_arrays)

    out_meta = ds.meta
    
    return img_stacked, out_meta

In [6]:
def caclulate_statistics_summary(img_stacked):
    # This function calculates statistical values by pixel
    
    img_stat_dict = {}
    statistics_list = ['mean'] #['mean','diff','std']
    for stat in statistics_list:
        if stat == 'mean':
            img_stat = img_stacked.mean(axis=0)
        elif stat == 'diff':
            img_stat = img_stacked.max(axis=0) - img_stacked.min(axis=0)
        elif stat == 'std':
            img_stat = img_stacked.std(axis=0)
        
        img_stat = img_stat.reshape((1,img_stat.shape[0],img_stat.shape[1]))
        img_stat_dict[stat] = img_stat
    
    return img_stat_dict

In [8]:
def save_summary_raster(img_stat_dict, out_meta):
    # This function saves the resulting rasters as .tif
    
    print(f'{raster_band} - Saving image.')
    
    for stat in img_stat_dict.keys():
        with ro.open(f"{dir_grl}filtered_dates/{raster_band}_{stat}.tif",'w', **out_meta) as dest:
            dest.write(img_stat_dict[stat])
            
            dest.close()

## __Run functions by band__

In [10]:
for raster_band in bands_to_join:
    print(f'--- Started processing {raster_band}.')
    img_stacked, out_meta = stacked_arrays(raster_band)
    img_stat_dict = caclulate_statistics_summary(img_stacked)
    save_summary_raster(img_stat_dict, out_meta)
    print(f'--- Finished processing {raster_band}.')

--- Started processing B02.
B02 - Stacking arrays.
B02 - Saving image.
--- Finished processing B02.
--- Started processing B03.
B03 - Stacking arrays.
B03 - Saving image.
--- Finished processing B03.
--- Started processing B04.
B04 - Stacking arrays.
B04 - Saving image.
--- Finished processing B04.
--- Started processing B05.
B05 - Stacking arrays.
B05 - Saving image.
--- Finished processing B05.
--- Started processing B06.
B06 - Stacking arrays.
B06 - Saving image.
--- Finished processing B06.
--- Started processing B07.
B07 - Stacking arrays.
B07 - Saving image.
--- Finished processing B07.
--- Started processing B8A.
B8A - Stacking arrays.
B8A - Saving image.
--- Finished processing B8A.
--- Started processing B11.
B11 - Stacking arrays.
B11 - Saving image.
--- Finished processing B11.
--- Started processing B12.
B12 - Stacking arrays.
B12 - Saving image.
--- Finished processing B12.
